# HEAD AND SHOULDERS YOUTUBE CODE

In [ ]:
df = pd.read_csv('FX_EURUSD, 1D.csv')

In [ ]:
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [ ]:
# df = pd.read_csv('/Users/tengliang/Documents/GitHub/head_shoulder_bots/data/bitcoin/btc_1h_2015_10_08.csv', 
#                          parse_dates=['date'],
#                          index_col=['date'],
#                          header=1
#                         )

#df_list = ["FX_EURUSD, 1D.csv","FX_EURUSD, 1200.csv","FX_EURUSD, 960.csv","FX_EURUSD, 720.csv","FX_EURUSD, 480.csv","FX_EURUSD, 240.csv","FX_EURUSD, 60.csv",]
#df_list = ["BATS_SPY, 60.csv","BATS_SPY, 240.csv","BATS_SPY, 480.csv","BATS_SPY, 720.csv","BATS_SPY, 960.csv","BATS_SPY, 1200.csv","BATS_SPY, 1D (7).csv",]
#df_list = ["FX_USDJPY, 1D.csv","FX_USDJPY, 1200.csv","FX_USDJPY, 960.csv","FX_USDJPY, 720.csv","FX_USDJPY, 480.csv","FX_USDJPY, 240.csv","FX_USDJPY, 60.csv",]
#df_list = ["FX_GBPUSD, 1D.csv","FX_GBPUSD, 1200.csv","FX_GBPUSD, 960.csv","FX_GBPUSD, 720.csv","FX_GBPUSD, 480.csv","FX_GBPUSD, 240.csv","FX_GBPUSD, 60.csv",]
#df_list = ["FX_AUDUSD, 1D.csv","FX_AUDUSD, 1200.csv","FX_AUDUSD, 960.csv","FX_AUDUSD, 720.csv","FX_AUDUSD, 480.csv","FX_AUDUSD, 240.csv","FX_AUDUSD, 60.csv",]
df_list = ["FX_USDCAD, 1D.csv","FX_USDCAD, 1200.csv","FX_USDCAD, 960.csv","FX_USDCAD, 720.csv","FX_USDCAD, 480.csv","FX_USDCAD, 240.csv","FX_USDCAD, 60.csv",]

hs_size = {"big" : [45,15], "medium" : [15,5], "small":[5,2]}
#df = pd.read_csv('FX_EURUSD, 1D.csv')
#df = pd.read_csv('FX_EURUSD, 1200.csv')
#df = pd.read_csv('FX_EURUSD, 240.csv')
#df = pd.read_csv('btc_1d_regular.csv')
# Remove the MultiIndex header and reset the column names
#df.columns = df.iloc[0]
#df = df[1:].reset_index(drop=True)

#df = df.rename(columns={'Volume BTC': 'volume','date':'time'})

In [ ]:
!pip install plotly

In [ ]:
!pip install -U kaleido

In [ ]:
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import os

In [ ]:
for size in hs_size:
    print(hs_size[size][0])

In [ ]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
import numpy as np

def pointpos(x):
    if x['pivot'] == 1:
        return x['low'] - 1e-3
    elif x['pivot'] == 2:
        return x['high'] + 1e-3
    else:
        return np.nan

def shortpointpos(x):
    if x['shortpivot'] == 1:
        return x['low'] - 2e-3
    elif x['shortpivot'] == 2:
        return x['high'] + 2e-3
    else:
        return np.nan

#### the below function plots and saves head and shoulder as png

In [ ]:
count = 0
for size in hs_size:
    for stock_df in df_list:
        df = pd.read_csv(stock_df)
        df['pivot'] = df.apply(lambda x: pivotid(df, x.name,hs_size[size][0],hs_size[size][0]), axis=1)
        df['shortpivot'] = df.apply(lambda x: pivotid(df, x.name,hs_size[size][1],hs_size[size][1]), axis=1)
        df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
        df['shortpointpos'] = df.apply(lambda row: shortpointpos(row), axis=1)
        import matplotlib.pyplot as plt
        from scipy.stats import linregress

        backcandles= hs_size[size][0]-1 #!!!should be less than pivot candles

        for candleid in range(0, len(df)-backcandles):
            if df.iloc[candleid].pivot != 2 or df.iloc[candleid].shortpivot != 2:
                continue

            maxim = np.array([])
            minim = np.array([])
            xxmin = np.array([])
            xxmax = np.array([])
            minbcount=0 #minimas before head
            maxbcount=0 #maximas before head
            minacount=0 #minimas after head
            maxacount=0 #maximas after head

            for i in range(candleid-backcandles, candleid+backcandles):
                if df.iloc[i].shortpivot == 1:
                    minim = np.append(minim, df.iloc[i].low)
                    xxmin = np.append(xxmin, i)
                    if i < candleid:
                        minbcount=+1
                    elif i>candleid:
                        minacount+=1
                if df.iloc[i].shortpivot == 2:
                    maxim = np.append(maxim, df.iloc[i].high)
                    xxmax = np.append(xxmax, i)
                    if i < candleid:
                        maxbcount+=1
                    elif i>candleid:
                        maxacount+=1

            if minbcount<1 or minacount<1 or maxbcount<1 or maxacount<1:
                continue

            slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)

            headidx = np.argmax(maxim, axis=0)
            if maxim[headidx]-maxim[headidx-1]>1.5e-3 and maxim[headidx]-maxim[headidx+1]>1.5e-3 and xxmin[0]>xxmax[headidx-1] and xxmin[1]<xxmax[headidx+1]:# and (maxim[headidx]-maxim[headidx+1])>(maxim[headidx+1]-minim[headidx+1]) and (maxim[headidx]-maxim[headidx-1])>(maxim[headidx-1]-minim[headidx-1]) :
                dfpl = df[candleid-backcandles: candleid+backcandles]
                plt.plot(dfpl.index, dfpl['close'])

                # Remove tick labels
                plt.gca().set_xticklabels([])
                plt.gca().set_yticklabels([])

                # Remove tick marks
                plt.gca().tick_params(axis='x', length=0)
                plt.gca().tick_params(axis='y', length=0)

                # Replace 'hs' with the desired folder name
                folder_name = 'hs'

                # Check if the folder exists, if not, create it
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)

                folder_path = os.path.abspath(folder_name)
                print(f"Folder path: {folder_path}")

                # Save the plot to the folder
                file_path = f'{folder_name}/closing_price_lineplot_{stock_df}_{size}_{candleid}.png'
                plt.savefig(file_path, dpi=50)
                print(f"File saved at: {os.path.abspath(file_path)}")
                print(f"SIZE IS {size}")
                print(f"count is {count}")
                plt.show()

                count+=1

                fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                open=dfpl['open'],
                                high=dfpl['high'],
                                low=dfpl['low'],
                                close=dfpl['close'])])
                fig.update_layout(
                    xaxis_rangeslider_visible=False,
                    xaxis=dict(
                        showticklabels=False  # Remove x-axis tick labels (numbers)
                    ),
                    yaxis=dict(
                        showticklabels=False  # Remove y-axis tick labels (numbers)
                    )
                )
                fig.show()
                # Replace the folder path below with your desired folder path
                folder_path = "hs/"
                file_name = f"candlestick_plot_{stock_df}_{size}_{candleid}.png"
                full_path = folder_path + file_name
                pio.write_image(fig, full_path, format="png")


            if candleid % 1000 == 0:
                print(candleid)


In [ ]:
!zip -r hs.zip hs


In [ ]:
from IPython.display import FileLink
FileLink('hs.zip')


#### Below plots non-head and shoulder patterns:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import plotly.graph_objs as go
import plotly.io as pio

# ... (all function definitions remain the same)

count = 0
head_and_shoulders_ranges = {}
for size in hs_size:
    for stock_df in df_list:
        df = pd.read_csv(stock_df)
        df['pivot'] = df.apply(lambda x: pivotid(df, x.name,hs_size[size][0],hs_size[size][0]), axis=1)
        df['shortpivot'] = df.apply(lambda x: pivotid(df, x.name,hs_size[size][1],hs_size[size][1]), axis=1)
        df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
        df['shortpointpos'] = df.apply(lambda row: shortpointpos(row), axis=1)
        import matplotlib.pyplot as plt
        from scipy.stats import linregress

        backcandles = hs_size[size][0] - 1

        for candleid in range(0, len(df) - backcandles):
            if df.iloc[candleid].pivot != 2 or df.iloc[candleid].shortpivot != 2:
                continue

            maxim = np.array([])
            minim = np.array([])
            xxmin = np.array([])
            xxmax = np.array([])
            minbcount=0 #minimas before head
            maxbcount=0 #maximas before head
            minacount=0 #minimas after head
            maxacount=0 #maximas after head

            for i in range(candleid-backcandles, candleid+backcandles):
                if df.iloc[i].shortpivot == 1:
                    minim = np.append(minim, df.iloc[i].low)
                    xxmin = np.append(xxmin, i)
                    if i < candleid:
                        minbcount=+1
                    elif i>candleid:
                        minacount+=1
                if df.iloc[i].shortpivot == 2:
                    maxim = np.append(maxim, df.iloc[i].high)
                    xxmax = np.append(xxmax, i)
                    if i < candleid:
                        maxbcount+=1
                    elif i>candleid:
                        maxacount+=1

            if minbcount<1 or minacount<1 or maxbcount<1 or maxacount<1:
                continue

            slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)

            headidx = np.argmax(maxim, axis=0)

            if maxim[headidx]-maxim[headidx-1]>1.5e-3 and maxim[headidx]-maxim[headidx+1]>1.5e-3 and xxmin[0]>xxmax[headidx-1] and xxmin[1]<xxmax[headidx+1]:
                # Add the range of indices for the detected head and shoulders pattern
                if stock_df not in head_and_shoulders_ranges:
                    head_and_shoulders_ranges[stock_df] = []
                head_and_shoulders_ranges[stock_df].append(range(candleid - backcandles, candleid + backcandles))



# Flatten the list of ranges and convert to a set for faster lookups
head_and_shoulders_indices = set(index for r in head_and_shoulders_ranges for index in r)



In [ ]:
head_and_shoulders_ranges

In [ ]:
count=0
for size in hs_size:
    for stock_df in df_list:
        df = pd.read_csv(stock_df)
        
        backcandles = hs_size[size][0] - 1
        
        non_hs_ranges = []
        if stock_df in head_and_shoulders_ranges:
            # Get the non-head and shoulders ranges by subtracting all head and shoulders ranges from the full range of the df
            full_range = set(range(0, len(df)))
            hs_ranges = head_and_shoulders_ranges[stock_df]
            for hs_range in hs_ranges:
                full_range -= set(range(hs_range[0], hs_range[1]+1)) # Add 1 to include the end index
            non_hs_ranges = list(full_range)

        # Loop through the non-head and shoulders ranges
        candle_start = 0
        print(non_hs_ranges)
        for candleid in non_hs_ranges:
            if candleid >= candle_start and candleid > 0+backcandles:
                print(f'Back candles left is {candleid-backcandles}')
                print(f'Back candles right is {candleid+backcandles}')
                if (candleid-backcandles) in non_hs_ranges and (candleid+backcandles) in non_hs_ranges:
                    # Plot the non-head and shoulders patterns
                    dfpl = df[candleid-backcandles: candleid+backcandles]

                    plt.plot(dfpl.index, dfpl['close'])

                    # Remove tick labels
                    plt.gca().set_xticklabels([])
                    plt.gca().set_yticklabels([])

                    # Remove tick marks
                    plt.gca().tick_params(axis='x', length=0)
                    plt.gca().tick_params(axis='y', length=0)

                    # Replace 'hs' with the desired folder name
                    folder_name = 'non_hs'

                    # Check if the folder exists, if not, create it
                    if not os.path.exists(folder_name):
                        os.makedirs(folder_name)

                    folder_path = os.path.abspath(folder_name)
                    print(f"Folder path: {folder_path}")

                    # Save the plot to the folder
                    file_path = f'{folder_name}/closing_price_lineplot_{stock_df}_{size}_{candleid}.png'
                    plt.savefig(file_path, dpi=50)
                    print(f"File saved at: {os.path.abspath(file_path)}")
                    print(f"SIZE IS {size}")
                    print(f"count is {count}")
                    plt.show()
                    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                    open=dfpl['open'],
                                    high=dfpl['high'],
                                    low=dfpl['low'],
                                    close=dfpl['close'])])
                    fig.update_layout(
                        xaxis_rangeslider_visible=False,
                        xaxis=dict(
                            showticklabels=False  # Remove x-axis tick labels (numbers)
                        ),
                        yaxis=dict(
                            showticklabels=False  # Remove y-axis tick labels (numbers)
                        )
                    )
                    fig.show()
                    # Replace the folder path below with your desired folder path
                    folder_path = "non_hs/"
                    file_name = f"candlestick_plot_{stock_df}_{size}_{candleid}.png"
                    full_path = folder_path + file_name
                    pio.write_image(fig, full_path, format="png")

                    count+=1
                    candle_start = candleid+backcandles


In [ ]:
!zip -r non_hs.zip non_hs
